In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, model_from_json



classes = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9'
}


name = 'model_digits_1'                                 # CHANGE NAME OF MODEL HERE
filename = name + ".hdf5"
model = load_model(filename, compile=False)
np.resize(model, 10)



# from https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
# function to resize + pad hand image
def get_square(image,square_size):

    height,width=image.shape
    if(height>width):
      differ=height
    else:
      differ=width
    differ+=4

    mask = np.zeros((differ,differ), dtype="uint8")   
    x_pos=int((differ-width)/2)
    y_pos=int((differ-height)/2)
    mask[y_pos:y_pos+height,x_pos:x_pos+width]=image[0:height,0:width]
    mask=cv2.resize(mask,(square_size,square_size),interpolation=cv2.INTER_AREA)

    return mask 





cap = cv2.VideoCapture(1)
rotate = 0
yes_hand = 0                                                  # for determining whether to display hand or not

while(True):
    
    _, frame = cap.read()                                           # Capture frame-by-frame
    
    frame = cv2.medianBlur(frame,5)                                 # apply noise/high-F filter
    
    if rotate == 1:                                                   # rotate frame by 180
        (h,w) = frame.shape[:2]
        center = (w/2, h/2)
        M = cv2.getRotationMatrix2D(center,180,1)
        frame = cv2.warpAffine(frame, M, (w,h))
    
    frameGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)             # Convert BGR to GRAYSCALE
    
   


    #### Filtering using 1) grayness 2) size ####
     
    lower_glove = 0                                                              # filtering for dark 
    upper_glove = 10
    frameHand = cv2.inRange(frameGray, lower_glove, upper_glove) 
    
    kernelCLOSE = np.ones((5,5), np.uint8)                                       # for thresholding unwanted "noise"
    frameHand = cv2.morphologyEx(frameHand, cv2.MORPH_CLOSE, kernelCLOSE)             # remove black stuff, in background
    frameHand2 = frameHand.copy()  
    
    _, contours, _ = cv2.findContours(frameHand,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_TC89_L1)       # run contours algorithm
    numContours = range(len(contours))
    centers = []
    numCenters = 0
    
    for i in numContours:                                       # find actual contours
        if cv2.contourArea(contours[i]) < 30000:                # do not consider noise
            continue
        if cv2.contourArea(contours[i]) > 60000:                # do not consider large objects
            continue
        M = cv2.moments(contours[i])                            # calculate moments for each contour
        if M["m00"] != 0:                                       # calculate the center of each object
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0    
        centers.append((cX,cY))
        x,y,w,h = cv2.boundingRect(contours[i])
        cv2.circle(frameHand, centers[-1], 4, (200, 200, 255), -1)      # centroids shown in gray
        cv2.rectangle(frameHand,(x,y),(x+w,y+h),(200, 200, 255), 2)     # rectangle bound
        cv2.circle(frame, centers[-1], 4, (200, 200, 255), -1)      # centroids shown in gray
        cv2.rectangle(frame,(x,y),(x+w,y+h),(200, 200, 255), 2)     # rectangle bound
        
    numCenters = len(centers)   
    if (numCenters == 1):                                       # create separate image for hand
        yes_hand = 1
        hand = frameHand2[y:y+h, x:x+w]                       
        hand = get_square(hand, 200)                            # resize + pad image to get same size image
        hand_crop_resized = np.expand_dims(cv2.resize(hand, (54, 54)), axis=0).reshape((1, 54, 54, 1))
    else:
        yes_hand = 0 
        
        
            
    #### Display results ####
    #### Predict hand ####
    
    display = frame.copy() 
    data_display = np.zeros_like(display, dtype=np.uint8)     # Black screen to display data
    positions = {                                             # Display positions (pixel coordinates)
        'hand_pose': (15, 400), # hand pose text
        'fps': (15, 20), # fps counter
        'null_pos': (200, 200) # used as null point for mouse control
    }
    

    cv2.imshow('frameHand', frameHand)
    cv2.imshow('frame', frame)
    
    
    if (yes_hand == 1):
        prediction = model.predict(hand_crop_resized)
        predi = prediction[0].argmax()             # get index of greatest confidence
        gesture = classes[predi]                   # identify gesture
        
        for i, pred in enumerate(prediction[0]):
            # Draw confidence bar for each gesture
            barx = positions['hand_pose'][0]
            bary = 60 + i*60
            bar_height = 20
            bar_length = int(400 * pred) + barx # calculate length of confidence bar
            
            # Make the most confidence prediction green
            if i == predi:
                colour = (0, 255, 0)
            else:
                colour = (0, 0, 255)
            
            cv2.putText(data_display, "{}: {}".format(classes[i], pred), (positions['hand_pose'][0], 30 + i*60), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
            cv2.rectangle(data_display, (barx, bary), (bar_length, bary - bar_height), colour, -1, 1)
            cv2.putText(data_display, "hand pose: {}".format(gesture), positions['hand_pose'], cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
        
        cv2.imshow('data', data_display)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):        # press 'q' to quit
        break

        
cap.release()
cv2.destroyAllWindows()